In [1]:
import pandas as pd
import os
import numpy as np
import sys

In [ ]:
# file_list = os.listdir(os.getcwd()+'/dataset (csv) labelled')

In [ ]:
# doc_list = [file for file in file_list if file.endswith('.csv')]

In [ ]:
#edit timestamp untuk chat7.csv
# chat7 = pd.read_csv(
#     'dataset (csv) labelled/chat7.csv',
#     usecols=['timestamp', 'sender', 'messages', 'sentiment_per_chat', 'sentiment_per_sesi']
# )
# chat7 = chat7.values.tolist()
# new_chat7 = list()
# for i in chat7:
#     each = list()
#     for j, doc in enumerate(i):
#         if j == 0:
#             doc = str(doc)[0:5]
#         each.append(doc)
#     new_chat7.append(each)

In [ ]:
# new_chat7_np = np.array(new_chat7)

In [ ]:
#save new_chat7 as csv
# chat7_df = pd.DataFrame(
#     list(zip(new_chat7_np[:, 0], new_chat7_np[:, 1], new_chat7_np[:, 2], new_chat7_np[:, 3], new_chat7_np[:, 4])),
#     columns=['timestamp', 'sender', 'messages', 'sentiment_per_chat', 'sentiment_per_sesi']
# )
# chat7_df.to_csv('chat7_new.csv', index=False)

In [ ]:
#edit timestamp untuk chat10.csv
# chat10 = pd.read_csv(
#     'dataset (csv) labelled/chat10.csv',
#     usecols=['timestamp', 'sender', 'messages', 'sentiment_per_chat', 'sentiment_per_sesi']
# )
# chat10 = chat10.values.tolist()
# new_chat10 = list()
# for i in chat10:
#     each = list()
#     for j, doc in enumerate(i):
#         if j == 0:
#             doc = doc.replace('[', '')
#             doc = doc.replace(']', '')
#             doc = doc.split(' ')
#             doc = doc[0].replace('/', '')
#         each.append(doc)
#     new_chat10.append(each)

In [ ]:
# new_chat10_np = np.array(new_chat10)

In [ ]:
#save new_chat10 as csv
# chat10_df = pd.DataFrame(
#     list(zip(new_chat10_np[:, 0], new_chat10_np[:, 1], new_chat10_np[:, 2], new_chat10_np[:, 3], new_chat10_np[:, 4])),
#     columns=['timestamp', 'sender', 'messages', 'sentiment_per_chat', 'sentiment_per_sesi']
# )
# chat10_df.to_csv('chat10_new.csv', index=False)

In [39]:
def parsing_csv(nama_file):
    chats = pd.read_csv(
        nama_file,
        usecols=["timestamp", "sender", "messages", "sentiment_per_chat", "sentiment_per_sesi"]
    )
    chats = chats.values.tolist()
    
    chats = remove_newline_in_message(chats)
    
    sessioned_chats = grouping_chat_per_session(chats)
    
    sessioned_chats_by_sender = grouping_chat_per_sender(sessioned_chats)
    
    write_into_txt(nama_file, sessioned_chats_by_sender)

In [40]:
parsing_csv('chat7_new.csv')
parsing_csv('chat10_new.csv')

chat7_new.csv file has been parsed!
chat10_new.csv file has been parsed!


In [27]:
def remove_newline_in_message(chats):
    hasil = list()
    for chat in chats:
        chat[2] = chat[2].replace('\n', ' ')
        hasil.append(chat)
    return hasil

In [32]:
def grouping_chat_per_session(chats):
    hasil = list()
    per_sesi = list()
    date_bfr = chats[0][0]
    for i, chat in enumerate(chats):
        if (chat[0] != date_bfr):
            sentimen = chats[i-1][4]
            per_sesi[-1][2] = sentimen
            hasil.append(per_sesi)
            per_sesi = list()
            date_bfr = chat[0]
        chat_per_sesi = [chat[1], chat[2], chat[3]]
        per_sesi.append(chat_per_sesi)
        if (i == len(chats)-1):
            if (len(per_sesi) == 1):
                sentimen = chats[i][4]
            else:
                sentimen = chats[i-1][4]
            per_sesi[-1][2] = sentimen
            hasil.append(per_sesi)
    return hasil

In [33]:
def find_same_sender_idx(sesi):
    same_index = list()
    sender_idx_bfr = -1
    sender_idx_aft = -1
    for i, chat in enumerate(sesi):
        if (i == len(sesi)-1):
            if (sender_idx_bfr != -1 and sender_idx_aft != -1):
                    same_index.append([sender_idx_bfr, sender_idx_aft])
                    sender_idx_bfr = -1
                    sender_idx_aft = -1
        else:
            if (sesi[i][0] == sesi[i+1][0]):
                sender_idx_bfr = i
                sender_idx_aft = i+1
            else:
                if (sender_idx_bfr != -1 and sender_idx_aft != -1):
                    same_index.append([sender_idx_bfr, sender_idx_aft])
                    sender_idx_bfr = -1
                    sender_idx_aft = -1
    return same_index

In [34]:
def grouping_same_idx(sesi, same_idx):
    not_incl = list()
    sesi_copy = sesi[:]
    for idx in same_idx:
        message = ""
        for i, x in enumerate(idx):
            message += sesi_copy[x][1] + " "
            if (i != 0): not_incl.append(x)
        sesi_copy[idx[0]][1] = message
        sesi_copy[idx[0]][2] = sesi[idx[-1]][2]
    
    new_sesi = list()
    for i, chat in enumerate(sesi_copy):
        if i not in not_incl:
            new_sesi.append(sesi_copy[i])
    return new_sesi

In [35]:
def grouping_chat_per_sender(sessioned_chats):
    sessioned_chats_by_sender = list()
    for i, sesi in enumerate(sessioned_chats):
        same_idx = find_same_sender_idx(sesi)
        if same_idx:
            sessioned_chats_by_sender.append(grouping_same_idx(sesi, same_idx))
        if not same_idx:
            sessioned_chats_by_sender.append(sesi)
    return sessioned_chats_by_sender

In [38]:
def write_into_txt(doc, sessioned_chats_by_sender):
    nama_folder = 'parsed doc new2/' + doc.replace('.csv', '')
    error = False
    try:
        os.makedirs(os.path.join(os.getcwd(), nama_folder))
    except OSError as error_msg:
        print(error_msg)
        error = True
    
    if not error:
        for i, sesi in enumerate(sessioned_chats_by_sender):
            text_per_sesi = ""
            for j, chat in enumerate(sesi):
                text_per_sesi += chat[0] + "\n"
                text_per_sesi += chat[1] + "\n"
                text_per_sesi += str(chat[2])
                if j != len(sesi)-1:
                    text_per_sesi += "\n"
            nama_file = nama_folder + '/sesi' + str(i) + '.txt'
            f = open(nama_file, 'w', encoding='utf8')
            f.write(text_per_sesi)
            f.close()
        print(doc + ' file has been parsed!')

In [ ]:
# def clean_date_format(chats):
#     for i, chat in enumerate(chats):
#         temp = chat[0].replace('[', '').replace(',', '')
#         temp = temp.split()[0]
#         chats[i][0] = temp
#     return chats